In [1]:
from geoband.API import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
import pandas as pd
import json
from pandas import json_normalize
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from geoband.API import *
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from geopandas import GeoDataFrame
# GetCompasData로 주피터 노트북 경로에 파일 저장
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string
from functools import reduce

In [2]:
others_old_house = gpd.read_file('7.4개시도_주택노후도.geojson')
others_alcohol = gpd.read_file('8.4개시도_유흥업소현황.geojson')
# others_use_area = gpd.read_file('9.4개시도_용도지역도.geojson')
# others_business_area = gpd.read_file('10.4개시도_사업지구경계도.geojson')
others_police_area = gpd.read_file('11.4개시도_경찰서_관할경계.geojson')
others_police_box_area = gpd.read_file('12.4개시도_치안안전센터_관할경계.geojson')
others_security_lamp = pd.read_csv('22.4개시도_보안등현황.csv')
others_security_bell = gpd.read_file('23.4개시도_안전비상벨현황.geojson')
others_cctv = pd.read_csv('24.4개시도_CCTV설치현황.csv')
others_park =pd.read_csv('27.4개시도_공원현황.csv')

In [34]:
def point_police(data_df,others_police_area,col):
    df = data_df.copy()

    epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    df['geometry'] = df.apply(lambda x : Point([x['lon'], x['lat']]), axis=1)
    df = gpd.GeoDataFrame(df, geometry='geometry', crs = epsg4326)
    
    police_result = gpd.sjoin(others_police_area, df, how='inner', op="intersects").drop(["index_right"],axis=1)
    
    police_result_groupby = police_result.groupby("관할경찰서").agg({col:"count"}).reset_index().rename(columns={col:"cnt"})
    
    return police_result,police_result_groupby

관할 경찰서 경계

In [4]:
others_police_area = gpd.read_file('11.4개시도_경찰서_관할경계.geojson')
others_police_area.columns = ["관할경찰서","관할경찰청","geometry"]
others_police_area= others_police_area[['관할경찰서','geometry']]

In [5]:
others_old_house.columns = ["원천공간정보ID","건물명칭","동명칭","필지고유번호","구조명","용도명","승인일자","건물연한","건물노후도","geometry"]
others_old_house = others_old_house[["구조명","용도명","건물연한","건물노후도","geometry"]]


others_alcohol = gpd.read_file('8.4개시도_유흥업소현황.geojson')[['BUSINESS_NM','COM_LC_NM','COM_MC_NM',
                'COM_SC_NM','KSIC_NM','SGG_NM',
       'ADB_EMD_NM','EMD_NM','ROAD_NM','geometry']]
others_alcohol.columns = ["상호명","상권업종대분류명","상권업종중분류명","상권업종소분류명","표준산업분류명",
                         "시군구명","행정동명","법정동명","도로명","geometry"]

# others_use_area.columns = ["관리번호","별칭","비고","고시일자","원천오브젝트ID","원천시군구코드","geometry"]
others_security_lamp.columns = ["설치년도","설치주소","설치대수","lon","lat"]
others_security_bell.columns = ["설치목적","설치장소유형","설치위치","설치년도","geometry"]
others_cctv.columns = ["CCTV설치위치","CCTV설치목적","CCTV설치대수","CCTV설치년월","lat","lon"]

최종 결합 police_result copy

In [6]:
police_old_house = gpd.sjoin(others_police_area, others_old_house, how='left', op="intersects")

In [7]:
police_old_house_groupby = police_old_house.groupby("관할경찰서").agg({"건물연한":"mean","건물노후도":"mean"}).reset_index()

생활관련 시설 count

In [8]:
police_용도명 = police_old_house.groupby(["관할경찰서","용도명"]).agg({"구조명":"count"}).reset_index().rename(columns={"구조명":"cnt"})
police_생활관련 = police_용도명[(police_용도명["용도명"] == "제1종근린생활시설") |
(police_용도명["용도명"] == "제2종근린생활시설") |
(police_용도명["용도명"] == "근린생활시설") |
(police_용도명["용도명"] == "단독주택") |
(police_용도명["용도명"] == "공동주택")].groupby("관할경찰서").agg({"cnt":"sum"}).reset_index()

In [9]:
police_용도명.groupby("용도명").agg({"cnt":"sum"}).reset_index().sort_values(by="cnt", ascending=False)

,용도명,cnt
9,단독주택,390812
1,공동주택,131670
27,제2종근린생활시설,68000
26,제1종근린생활시설,66972
2,공장,9585
18,업무시설,9353
5,교육연구시설,8591
29,창고시설,5457
8,노유자시설,4388
17,숙박시설,3510


In [10]:
# 많은거 상위 10개
police_old_house_groupby['단독주택']=0
police_old_house_groupby['공동주택']=0
police_old_house_groupby['제2종근린생활시설']=0
police_old_house_groupby['제1종근린생활시설']=0
police_old_house_groupby['공장']=0
police_old_house_groupby['업무시설']=0
police_old_house_groupby['교육연구시설']=0
police_old_house_groupby['창고시설']=0
police_old_house_groupby['노유자시설']=0
police_old_house_groupby['숙박시설']=0

In [11]:
for i in range(len(police_용도명)):
    for j in range(len(police_old_house_groupby)):
        if police_용도명.loc[i, '관할경찰서'] == police_old_house_groupby.loc[j, '관할경찰서'] and police_용도명.loc[i, '용도명'] == '단독주택':
            police_old_house_groupby.loc[j, '단독주택']=police_용도명.loc[i, 'cnt']
        elif police_용도명.loc[i, '관할경찰서'] == police_old_house_groupby.loc[j, '관할경찰서'] and police_용도명.loc[i, '용도명'] == '공동주택':
            police_old_house_groupby.loc[j, '공동주택']=police_용도명.loc[i, 'cnt']
        elif police_용도명.loc[i, '관할경찰서'] == police_old_house_groupby.loc[j, '관할경찰서'] and police_용도명.loc[i, '용도명'] == '제2종근린생활시설':
            police_old_house_groupby.loc[j, '제2종근린생활시설']=police_용도명.loc[i, 'cnt']
        elif police_용도명.loc[i, '관할경찰서'] == police_old_house_groupby.loc[j, '관할경찰서'] and police_용도명.loc[i, '용도명'] == '제1종근린생활시설':
            police_old_house_groupby.loc[j, '제1종근린생활시설']=police_용도명.loc[i, 'cnt']
        elif police_용도명.loc[i, '관할경찰서'] == police_old_house_groupby.loc[j, '관할경찰서'] and police_용도명.loc[i, '용도명'] == '공장':
            police_old_house_groupby.loc[j, '공장']=police_용도명.loc[i, 'cnt']
        elif police_용도명.loc[i, '관할경찰서'] == police_old_house_groupby.loc[j, '관할경찰서'] and police_용도명.loc[i, '용도명'] == '업무시설':
            police_old_house_groupby.loc[j, '업무시설']=police_용도명.loc[i, 'cnt']
        elif police_용도명.loc[i, '관할경찰서'] == police_old_house_groupby.loc[j, '관할경찰서'] and police_용도명.loc[i, '용도명'] == '교육연구시설':
            police_old_house_groupby.loc[j, '교육연구시설']=police_용도명.loc[i, 'cnt']
        elif police_용도명.loc[i, '관할경찰서'] == police_old_house_groupby.loc[j, '관할경찰서'] and police_용도명.loc[i, '용도명'] == '창고시설':
            police_old_house_groupby.loc[j, '창고시설']=police_용도명.loc[i, 'cnt']
        elif police_용도명.loc[i, '관할경찰서'] == police_old_house_groupby.loc[j, '관할경찰서'] and police_용도명.loc[i, '용도명'] == '노유자시설':
            police_old_house_groupby.loc[j, '노유자시설']=police_용도명.loc[i, 'cnt']
        elif police_용도명.loc[i, '관할경찰서'] == police_old_house_groupby.loc[j, '관할경찰서'] and police_용도명.loc[i, '용도명'] == '숙박시설':
            police_old_house_groupby.loc[j, '숙박시설']=police_용도명.loc[i, 'cnt']

In [12]:
police_old_house_groupby

,관할경찰서,건물연한,건물노후도,단독주택,공동주택,제2종근린생활시설,제1종근린생활시설,공장,업무시설,교육연구시설,창고시설,노유자시설,숙박시설
0,마산동부경찰서,34.221704,3.520023,14699,1126,1355,1962,762,101,148,229,120,209
1,마산중부경찰서,27.137778,3.040000,8489,591,1234,1735,387,88,109,484,121,155
2,서울강남경찰서,26.375468,3.038648,3533,2509,3606,1761,0,877,133,8,46,82
3,서울강동경찰서,26.086668,3.049578,9171,5101,1465,1531,5,210,179,30,153,86
4,서울강북경찰서,31.807006,3.248836,16671,5554,1340,2191,8,102,193,13,125,120
5,서울강서경찰서,27.005737,3.041989,10355,7842,1934,1983,88,291,298,111,209,167
6,서울관악경찰서,26.835200,3.006179,18389,6148,2990,2267,4,536,488,9,142,162
7,서울광진경찰서,27.653790,3.069064,14874,4733,1861,2053,11,226,305,12,128,97
8,서울구로경찰서,29.174967,3.178105,9876,4768,1241,1715,255,213,127,124,128,82
9,서울금천경찰서,28.568851,3.189752,8022,2849,1063,952,516,200,146,77,115,57


In [13]:
police_생활관련.columns = ['관할경찰서','생활관련시설']

8.4개시도_유흥업소현황.geojson

In [14]:
others_alcohol

,상호명,상권업종대분류명,상권업종중분류명,상권업종소분류명,표준산업분류명,시군구명,행정동명,법정동명,도로명,geometry
0,채플린노래연습장,관광/여가/오락,무도/유흥/가무,노래방,노래연습장 운영업,수원시 권선구,평동,오목천동,경기도 수원시 권선구 삼천병마로1598번길,POINT (126.96429 37.24533)
1,워커힐노래방,관광/여가/오락,무도/유흥/가무,노래방,노래연습장 운영업,수원시 장안구,영화동,영화동,경기도 수원시 장안구 팔달로271번길,POINT (127.01367 37.29094)
2,왈츠노래방,관광/여가/오락,무도/유흥/가무,노래방,노래연습장 운영업,수원시 팔달구,인계동,인계동,경기도 수원시 팔달구 효원로291번길,POINT (127.03431 37.26251)
3,락휴,관광/여가/오락,무도/유흥/가무,노래방,노래연습장 운영업,수원시 영통구,원천동,원천동,경기도 수원시 영통구 아주로,POINT (127.04413 37.27638)
4,현대노래연습장,관광/여가/오락,무도/유흥/가무,노래방,노래연습장 운영업,수원시 권선구,구운동,구운동,경기도 수원시 권선구 구운로,POINT (126.97484 37.27693)
...,...,...,...,...,...,...,...,...,...,...
4248,오케이노래연습장,관광/여가/오락,무도/유흥/가무,노래방,노래연습장 운영업,수원시 장안구,연무동,연무동,경기도 수원시 장안구 연무로42번길,POINT (127.02799 37.29635)
4249,별빛노래방,관광/여가/오락,무도/유흥/가무,노래방,노래연습장 운영업,수원시 권선구,권선1동,권선동,경기도 수원시 권선구 권선로640번길,POINT (127.02084 37.26014)
4250,인연노래연습장,관광/여가/오락,무도/유흥/가무,노래방,노래연습장 운영업,수원시 권선구,세류2동,장지동,경기도 수원시 권선구 정조로,POINT (127.01397 37.24464)
4251,일번가노래방,관광/여가/오락,무도/유흥/가무,노래방,노래연습장 운영업,수원시 장안구,조원1동,조원동,경기도 수원시 장안구 금당로,POINT (127.01404 37.30477)


In [15]:
police_alcohol,police_alcohol_groupby = point_police(others_alcohol,others_police_area,"상호명")

In [16]:
police_alcohol_groupby.columns=['관할경찰서','유흥업소']

22.4개시도_보안등현황.csv

In [17]:
police_security_lamp, police_security_lamp_groupby = point_police(others_security_lamp, others_police_area,'geometry')

In [18]:
police_security_lamp_groupby = police_security_lamp.groupby('관할경찰서').agg({'설치대수':'sum'}).reset_index()

In [19]:
police_security_lamp_groupby.columns = ['관할경찰서','보안등']

In [20]:
# 동대문구, 양천구 데이터 없음
police_security_lamp_groupby.head()

,관할경찰서,보안등
0,마산동부경찰서,5647
1,마산중부경찰서,6821
2,서울강남경찰서,5029
3,서울강동경찰서,7500
4,서울강북경찰서,8431


23.4개시도_안전비상벨현황.geojson

In [21]:
police_security_bell,police_security_bell_groupby = point_police(others_security_bell,others_police_area,'geometry')

In [22]:
police_security_bell_gb=police_security_bell.groupby(['관할경찰서', '설치장소유형']).agg({'설치위치':'count'}).reset_index()
police_security_bell_gb

,관할경찰서,설치장소유형,설치위치
0,마산동부경찰서,화장실,11
1,마산중부경찰서,화장실,14
2,서울강남경찰서,가로변,642
3,서울강남경찰서,건물,1
4,서울강남경찰서,화장실,6
...,...,...,...
89,진주경찰서,가로변,611
90,진주경찰서,공원,152
91,진해경찰서,화장실,39
92,창원서부경찰서,화장실,11


In [23]:
police_security_bell_groupby['안전비상벨_가로변']=0
police_security_bell_groupby['안전비상벨_공원']=0
police_security_bell_groupby['안전비상벨_화장실']=0

In [24]:
for i in range(len(police_security_bell_gb)):
    for j in range(len(police_security_bell_groupby)):
        if police_security_bell_gb.loc[i, '관할경찰서'] == police_security_bell_groupby.loc[j, '관할경찰서'] and police_security_bell_gb.loc[i, '설치장소유형'] == '가로변':
            police_security_bell_groupby.loc[j, '안전비상벨_가로변']=police_security_bell_gb.loc[i, '설치위치']
        elif police_security_bell_gb.loc[i, '관할경찰서'] == police_security_bell_groupby.loc[j, '관할경찰서'] and police_security_bell_gb.loc[i, '설치장소유형'] == '공원':
            police_security_bell_groupby.loc[j, '안전비상벨_공원']=police_security_bell_gb.loc[i, '설치위치']
        elif police_security_bell_gb.loc[i, '관할경찰서'] == police_security_bell_groupby.loc[j, '관할경찰서'] and police_security_bell_gb.loc[i, '설치장소유형'] == '화장실':
            police_security_bell_groupby.loc[j, '안전비상벨_화장실']=police_security_bell_gb.loc[i, '설치위치']


In [25]:
police_security_bell_groupby.rename(columns={"cnt":"안전비상벨"}, inplace=True)

'24.4개시도_CCTV설치현황.csv'

In [26]:
police_cctv,police_cctv_groupby = point_police(others_cctv,others_police_area,"geometry")

In [27]:
police_cctv_groupby = police_cctv.groupby('관할경찰서').agg({'CCTV설치대수':'sum'}).reset_index()

In [28]:
police_cctv['CCTV설치목적'].value_counts()

생활방범      28708
다목적        4982
어린이보호      4165
교통단속       3118
쓰레기단속      1412
기타          657
재난재해        550
시설물관리       514
차량방범        437
교통정보수집      430
Name: CCTV설치목적, dtype: int64

In [29]:
# 상위 5개
police_cctv_groupby['CCTV설치목적_생활방범']=0
police_cctv_groupby['CCTV설치목적_어린이보호']=0
police_cctv_groupby['CCTV설치목적_교통단속']=0
police_cctv_groupby['CCTV설치목적_쓰레기단속']=0
police_cctv_groupby['CCTV설치목적_재난재해']=0

In [30]:
police_cctv_gb=police_cctv.groupby(['관할경찰서', 'CCTV설치목적']).agg({'CCTV설치위치':'count'}).reset_index()

In [31]:
for i in range(len(police_cctv_gb)):
    for j in range(len(police_cctv_groupby)):
        if police_cctv_gb.loc[i, '관할경찰서'] == police_cctv_groupby.loc[j, '관할경찰서'] and police_cctv_gb.loc[i, 'CCTV설치목적'] == '생활방범':
            police_cctv_groupby.loc[j, 'CCTV설치목적_생활방범']=police_cctv_gb.loc[i, 'CCTV설치위치']
        elif police_cctv_gb.loc[i, '관할경찰서'] == police_cctv_groupby.loc[j, '관할경찰서'] and police_cctv_gb.loc[i, 'CCTV설치목적'] == '어린이보호':
            police_cctv_groupby.loc[j, 'CCTV설치목적_어린이보호']=police_cctv_gb.loc[i, 'CCTV설치위치']
        elif police_cctv_gb.loc[i, '관할경찰서'] == police_cctv_groupby.loc[j, '관할경찰서'] and police_cctv_gb.loc[i, 'CCTV설치목적'] == '교통단속':
            police_cctv_groupby.loc[j, 'CCTV설치목적_교통단속']=police_cctv_gb.loc[i, 'CCTV설치위치']
        elif police_cctv_gb.loc[i, '관할경찰서'] == police_cctv_groupby.loc[j, '관할경찰서'] and police_cctv_gb.loc[i, 'CCTV설치목적'] == '쓰레기단속':
            police_cctv_groupby.loc[j, 'CCTV설치목적_쓰레기단속']=police_cctv_gb.loc[i, 'CCTV설치위치']
        elif police_cctv_gb.loc[i, '관할경찰서'] == police_cctv_groupby.loc[j, '관할경찰서'] and police_cctv_gb.loc[i, 'CCTV설치목적'] == '재난재해':
            police_cctv_groupby.loc[j, 'CCTV설치목적_재난재해']=police_cctv_gb.loc[i, 'CCTV설치위치']


In [32]:
police_cctv_groupby

,관할경찰서,CCTV설치대수,CCTV설치목적_생활방범,CCTV설치목적_어린이보호,CCTV설치목적_교통단속,CCTV설치목적_쓰레기단속,CCTV설치목적_재난재해
0,마산동부경찰서,1218,297,110,28,18,4
1,마산중부경찰서,911,243,70,0,18,7
2,서울강남경찰서,3095,796,0,196,0,2
3,서울강북경찰서,2299,1690,320,93,0,13
4,서울강서경찰서,2377,1040,118,100,25,0
5,서울관악경찰서,4825,0,0,0,0,0
6,서울광진경찰서,6459,2387,643,162,280,20
7,서울구로경찰서,3659,881,308,95,29,28
8,서울금천경찰서,4226,0,0,0,0,0
9,서울남대문경찰서,725,196,5,36,12,0


27.4개시도_공원현황.csv

In [35]:
police_park,police_park_groupby = point_police(others_park,others_police_area,"geometry")

In [36]:
police_park_groupby.columns = ['관할경찰서','공원']

In [37]:
merge_list=[police_old_house_groupby,police_생활관련,police_alcohol_groupby,police_security_lamp_groupby,
           police_security_bell_groupby,police_cctv_groupby] 
police_result = reduce(lambda left,right: pd.merge(left,right,how='outer',on = ['관할경찰서']),merge_list)
police_result.fillna(0,inplace=True)

In [38]:
police_result.head()

,관할경찰서,건물연한,건물노후도,단독주택,공동주택,제2종근린생활시설,제1종근린생활시설,공장,업무시설,교육연구시설,...,안전비상벨,안전비상벨_가로변,안전비상벨_공원,안전비상벨_화장실,CCTV설치대수,CCTV설치목적_생활방범,CCTV설치목적_어린이보호,CCTV설치목적_교통단속,CCTV설치목적_쓰레기단속,CCTV설치목적_재난재해
0,마산동부경찰서,34.221704,3.520023,14699,1126,1355,1962,762,101,148,...,11.0,0.0,0.0,11.0,1218.0,297.0,110.0,28.0,18.0,4.0
1,마산중부경찰서,27.137778,3.040000,8489,591,1234,1735,387,88,109,...,14.0,0.0,0.0,14.0,911.0,243.0,70.0,0.0,18.0,7.0
2,서울강남경찰서,26.375468,3.038648,3533,2509,3606,1761,0,877,133,...,649.0,642.0,0.0,6.0,3095.0,796.0,0.0,196.0,0.0,2.0
3,서울강동경찰서,26.086668,3.049578,9171,5101,1465,1531,5,210,179,...,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,서울강북경찰서,31.807006,3.248836,16671,5554,1340,2191,8,102,193,...,22.0,0.0,2.0,20.0,2299.0,1690.0,320.0,93.0,0.0,13.0


In [39]:
police_result.to_csv("전처리완료_선용.csv", index=False)